In [ ]:
import pandas as pd
# Load data
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
data = pd.read_csv('/content/gdrive/MyDrive/cars (1).csv')

In [ ]:
data.isnull().sum()

In [ ]:
data

In [ ]:
X=data.copy()
y=X.pop('MSRP')

In [ ]:
X

In [ ]:
y

In [ ]:
X['Vehicle Size'].value_counts()

First of all I will handle the missing values using imputation with different strategies :

In [ ]:

cols_with_missing=[col for col in X if X[col].isnull().any()]

cols_with_missing


In [ ]:
X[cols_with_missing].dtypes

In [ ]:
numeric_x=X.select_dtypes(exclude=['object'])

from sklearn.impute import SimpleImputer

my_imputer1=SimpleImputer(strategy='mean')
numeric_x_imputed=pd.DataFrame(my_imputer1.fit_transform(numeric_x))

In [ ]:
categorical_x=X.select_dtypes(exclude=['int64','float64'])

my_imputer2=SimpleImputer(strategy='most_frequent')
categorical_x_imputed=pd.DataFrame(my_imputer2.fit_transform(categorical_x))

In [ ]:
numerical_cols=[col for col in X if X[col].dtypes=='float64' or X[col].dtypes=='int64']
categorical_cols=[col for col in X if X[col].dtypes=='object']

In [ ]:
numeric_x_imputed.columns=numerical_cols
categorical_x_imputed.columns=categorical_cols

In [ ]:
numeric_x_imputed

In [ ]:
categorical_x_imputed

In [ ]:
X_new=pd.concat([numeric_x_imputed,categorical_x_imputed],axis=1)

In [ ]:
X_new

In [ ]:
X_new.isnull().sum().sum()

After filling the missing values , I will use some feature engineering aproaches ,starting with finding the MI :

In [ ]:
from sklearn.feature_selection import mutual_info_regression

In [ ]:

for colname in X_new.select_dtypes("object"):
    X_new[colname], _ = X_new[colname].factorize()

discrete_features = X_new.dtypes == int


In [ ]:
mi_scores = mutual_info_regression(X_new, y, discrete_features=discrete_features)
mi_scores = pd.Series(mi_scores, name="MI Scores", index=X_new.columns)
mi_scores = mi_scores.sort_values(ascending=False)

In [ ]:
mi_scores


In [ ]:
# I can create new feature here , I searched fow a way to combine city MPG and highway MPG
# so I will add a new feature
# you can ask a domain expert to find the formula, in my case I used Google


X_new['Combined_MPG']=  X_new['city mpg'] * .55 +  X_new['highway MPG'] * .45

In [ ]:
X_new['Combined_MPG']

In [ ]:
# I will do new MI serie, if my new feature gets high MI , I will keep it

for colname in X_new.select_dtypes("object"):
    X_new[colname], _ = X_new[colname].factorize()

discrete_features = X_new.dtypes == int



mi_scores = mutual_info_regression(X_new, y, discrete_features=discrete_features)
mi_scores = pd.Series(mi_scores, name="MI Scores", index=X_new.columns)
mi_scores = mi_scores.sort_values(ascending=False)


mi_scores

In [ ]:
# another thing I can do is to remove 'number of doors' feature since it has the least MI
# which makes sense because you can conclude the number of doors by other features 'vehicle size' and 'vehicle style'


X_new.drop('Number of Doors',axis=1,inplace=True)

In [ ]:
X_new

Now I will build the model and then use MAE for validation :

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split


In [ ]:
def get_MAE(e):
  model=RandomForestRegressor(n_estimators=e,random_state=0)
  X_train,X_val,y_train,y_val=train_test_split(X_new,y,train_size=.8,test_size=.2,random_state=0)
  model.fit(X_train,y_train)
  yp=model.predict(X_val)
  error=mean_absolute_error(yp,y_val)
  return error



Now I will try the model parameters to find the least error   :

(I will try different number of estimators and then validate the model using the cross validation function)

In [ ]:
get_MAE(50)

In [ ]:
for i in [25,50,75,100]:
  print(get_MAE(i))

In [ ]:
for i in [100,200,300]:
  print(get_MAE(i))

In [ ]:
for i in [300,350,400]:
  print(get_MAE(i))

In [ ]:
#The last MAE before creating the final model
get_MAE(400)

In [ ]:
# The best number of estimators is 400

Now,I can build my final model and train it in all of the data !

In [ ]:

myFinalModel=RandomForestRegressor(n_estimators=400,random_state=0)
myFinalModel.fit(X_new,y)

In [ ]:
import pickle

filename = 'finalized_model.sav'
pickle.dump(myFinalModel, open(filename, 'wb'))